In [1]:
import requests

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver import ActionChains
import pandas as pd
import re
from datetime import datetime


In [2]:
def OpenNewBrowser(url):
    global driver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)

In [3]:
def OpenNewBrowserCentris(): 
    OpenNewBrowser("https://www.centris.ca/")

    understand = driver.find_element(By.CLASS_NAME, "term-use-banner__button-js.btn.btn-primary.term-use-banner__button")
    understand.click()


    search = driver.find_element(By.CLASS_NAME, "select2-search__field")
    search.send_keys("Montreal(Ile)")
    time.sleep(2)
    search.send_keys(Keys.ENTER)
    time.sleep(1)
    search.send_keys("Montreal(Rive-Sud)")
    time.sleep(2)
    search.send_keys(Keys.RETURN)
    time.sleep(1)
    search.send_keys("Montreal(Rive-Nord)")
    time.sleep(2)
    search.send_keys(Keys.RETURN)
    time.sleep(1)
    search.send_keys("Saint-Jean-sur-")
    time.sleep(2)
    search.send_keys(Keys.RETURN)
    time.sleep(1)
    search.send_keys("Saint-Remi")
    time.sleep(2)
    search.send_keys(Keys.RETURN)
    search.send_keys("McMasterville")
    time.sleep(2)
    search.send_keys(Keys.RETURN)
    time.sleep(1)
    search.send_keys("Beloeil")
    time.sleep(2)
    search.send_keys(Keys.RETURN)

    price = driver.find_element(By.ID, "SalePrice-button")
    price.click()
    time.sleep(1)
    move = ActionChains(driver)
    max = driver.find_element(By.CLASS_NAME, "slider-handle.max-slider-handle.round")
    move.click_and_hold(max).move_by_offset(-105, 0).release().perform()

    for b in driver.find_elements(By.CLASS_NAME, "btn.btn-primary.btn-search"):
        if b.text == "Appliquer":
            b.click()

    driver.find_element(By.ID, "filter-search").click()
    time.sleep(1)
    driver.find_element(By.ID, "BuildingSection-heading-filters").click()
    driver.find_element(By.ID, "YearConstruction-min").send_keys("1960")

    for b in driver.find_elements(By.CLASS_NAME, "btn.btn-link"):
        if b.text == "Type de propriété":
            b.click()

    for b in driver.find_elements(By.CLASS_NAME, "custom-control-label"):
        if b.text == "Plex":
            b.click()

    time.sleep(1)
    
    for b in driver.find_elements(By.CLASS_NAME, "btn.btn-link.collapsed"):
        if b.text == "Plex":
            b.click()
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 200)") 
    time.sleep(1)
    for b in driver.find_elements(By.CLASS_NAME, "custom-control-label"):
        if b.text == "Triplex":
            b.click()
        elif b.text == "Quadruplex":
            b.click()
        elif b.text == "Quintuplex":
            b.click()
    
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0)") 
    time.sleep(1)

    driver.find_element(By.ID, "js-home-button-search-container").click()

    return None 

In [4]:
OpenNewBrowserCentris()



====== WebDriver manager ======
Current google-chrome version is 112.0.5615
Get LATEST chromedriver version for 112.0.5615 google-chrome
Driver [C:\Users\Ricardo\.wdm\drivers\chromedriver\win32\112.0.5615.49\chromedriver.exe] found in cache


In [5]:
def GetCentrisLinks():
    time.sleep(5)
    TotalLinks = [] 

    pages = driver.find_element(By.CLASS_NAME, "pager-current").text
    pages = pages.replace(" ", "")
    current = int(pages.split("/")[0])
    total = int(pages.split("/")[1])
    #print(current)
    #print(total)

    while current <= total:
        time.sleep(5)
        try:
            listings = driver.find_elements(By.CLASS_NAME, "property-thumbnail-summary-link")
            links = [elem.get_attribute('href') for elem in listings]
            TotalLinks.extend(links)
            #print(links)
            #print(len(links))
            print(current/total)
            current += 1
            driver.find_element(By.CLASS_NAME, "next").click()

        except:
            driver.find_element(By.CLASS_NAME, "DialogInsightLightBoxCloseButton").click()

            listings = driver.find_elements(By.CLASS_NAME, "property-thumbnail-summary-link")
            links = [elem.get_attribute('href') for elem in listings]
            TotalLinks.append(links)
            #print(links)
            #print(len(links))
            current += 1
            driver.find_element(By.CLASS_NAME, "next").click()

    #TotalLinks = sorted(set(TotalLinks))

    #print(TotalLinks)
    print(len(TotalLinks))
    return TotalLinks



In [6]:
global ListingsURLs
ListingsURLs = GetCentrisLinks()

0.25
0.5
0.75
1.0
41


In [7]:
def readListing(listingsList):

    ListingsDF = pd.DataFrame(columns=['URL', 'Address', 'Price', 'PlexType', 'Usage', 'Year', 'GarageIn', 'GarageOut', 
                                       'Units', 'Revenues', 'Style', 'Sqft', 'NbUnits', 'MainUnit', 
                                       'AdditionalCaracts', 'Description'])
    count = 0

    for listing in listingsList:
        try:
            print(count/ len(listingsList))
            count += 1 
            if count == int(len(listingsList)/ 2):
                 ListingsDF.to_excel(r'C:\Users\Ricardo\Documents\Real Estate\Data\duplex_triplex' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.xlsx')
            #print(ListingsDF)
            driver.get(listing)
            time.sleep(3)
            print(listing)
            try:
                title = driver.find_element(By.CLASS_NAME, 'col.text-left.pl-0').text
            except:
                continue
            #print(title)
            
            #plexType = re.search('/(.*)~', listing).group(1)
            plexType = title.split("à vendre")[0]
            #print(plexType)
            address = driver.find_element(By.CLASS_NAME, 'pt-1').text
            price = int(driver.find_element(By.ID, "BuyPrice").text.replace(" ", "").replace("$", "")) 

            caracTitle = driver.find_elements(By.CLASS_NAME, "carac-title")
            caracValue = driver.find_elements(By.CLASS_NAME, "carac-value")

            usage = "null"
            year = "null"
            garageIn = "null"
            garageOut = "null"
            units = "null"
            revenues = "null"
            style = "null"
            sqft = "null"
            nbUnits = 0
            mainUnit = "null"
            addCarac = "null"
            desc = "null"

            for i in range(len(caracTitle)):
                if caracTitle[i].text == "Utilisation de la propriété":
                    usage = caracValue[i].text
                elif caracTitle[i].text == "Style de bâtiment":
                    style = caracValue[i].text
                elif caracTitle[i].text == "Année de construction":
                    try:
                        year = int(re.sub("[^0-9]", "", caracValue[i].text))
                    except:
                        year = "null"
                elif caracTitle[i].text == "Superficie du terrain":
                    sqft = int(caracValue[i].text.replace(" ", "").replace("pc", ""))
                elif caracTitle[i].text == "Stationnement total":
                    #print(caracValue[i].text)
                    try:
                        garageOut = (re.sub("[^0-9]", "", caracValue[i].text.split(",")[0]))
                        garageIn =  (re.sub("[^0-9]", "", caracValue[i].text.split(",")[1]))
                    except:
                        continue
                elif caracTitle[i].text == "Nombre d’unités":
                    nbUnits = int(re.sub("[^0-9]", "", caracValue[i].text))
                elif caracTitle[i].text == "Unités résidentielles":
                    units = caracValue[i].text
                elif caracTitle[i].text == "Unité principale":
                    mainUnit = caracValue[i].text
                elif caracTitle[i].text == "Revenus bruts potentiels":
                    revenues = int(caracValue[i].text.replace("$", "").replace(" ", ""))
                elif caracTitle[i].text == "Caractéristiques additionnelles":
                    addCarac = caracValue[i].text

            desc = driver.find_element(By.CLASS_NAME, "row.description-row").text
                
            # Filters
            if "commercial" in usage:
                continue
            """
            if not((nbUnits == 2 and price <= 500000) or \
                    (nbUnits == 3 and price <= 550000) or \
                    (nbUnits == 4 and price <= 600000) or \
                    (nbUnits == 5 and price <= 600000)):
                continue
            #if nbUnits > 2:
            #    print(listing)
            if year <= 1944:
                continue
            
            if ("Saint-Hyacinthe" in address) or ("Salaberry-de-Valleyfield" in address) or \
                ("Sorel-Tracy" in address) or ("Rigaud" in address) or \
                ("Acton Vale" in address) or ("Saint-Antoine-sur-Richelieu" in address):
                continue
            """
            ListingsDF = ListingsDF.append(pd.DataFrame([{
            'URL': listing,
            'Address': address,
            'Price': price,
            'PlexType': plexType,
            'Usage': usage,
            'Year': year, 
            'GarageIn': garageIn,
            'GarageOut': garageOut, 
            'Units': units, 
            'Revenues': revenues, 
            'Style': style, 
            'Sqft': sqft, 
            'NbUnits': nbUnits, 
            'MainUnit': mainUnit,
            'AdditionalCaracts':addCarac, 
            'Description': desc
            }]), ignore_index=True)

        except:
            continue
    
    print(ListingsDF)

    return ListingsDF
        

In [8]:
#OpenNewBrowserCentris()
#OpenNewBrowser("centris.com")
#listingsInfo = readListing(["https://www.centris.ca/fr/triplex~a-vendre~montreal-montreal-nord/13027121?view=Summary"])
listingsInfo = readListing(ListingsURLs)
listingsInfo.to_excel(r'C:\Users\Ricardo\Documents\Real Estate\Data\duplex_triplex' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.xlsx')

0.0
https://www.centris.ca/fr/triplex~a-vendre~sainte-anne-des-plaines/24865756?view=Summary
0.024390243902439025
https://www.centris.ca/fr/quadruplex~a-vendre~lavaltrie/23678983?view=Summary
0.04878048780487805
https://www.centris.ca/fr/quadruplex~a-vendre~saint-jean-sur-richelieu/27378574?view=Summary
0.07317073170731707
https://www.centris.ca/fr/triplex~a-vendre~saint-jerome/25868430?view=Summary
0.0975609756097561
https://www.centris.ca/fr/triplex~a-vendre~chateauguay/24248167?view=Summary
0.12195121951219512
https://www.centris.ca/fr/triplex~a-vendre~saint-jerome/10823511?view=Summary
0.14634146341463414
https://www.centris.ca/fr/triplex~a-vendre~la-prairie/24334867?view=Summary
0.17073170731707318
https://www.centris.ca/fr/triplex~a-vendre~montreal-mercier-hochelaga-maisonneuve/19608719?view=Summary
0.1951219512195122
https://www.centris.ca/fr/triplex~a-vendre~longueuil-saint-hubert/24727139?view=Summary
0.21951219512195122
https://www.centris.ca/fr/triplex~a-vendre~saint-remi/10

In [9]:
def compareHistorical(currentDF, historicalDF):

    newListings = currentDF[currentDF.URL.isin(historicalDF.URL) == False]

    return newListings

In [10]:
hist = pd.read_excel(r'C:\Users\Ricardo\Documents\Real Estate\Data\duplex_triplex2023_03_25_11_16_41.xlsx')

newDF = compareHistorical(listingsInfo, hist)

In [11]:
newDF

,URL,Address,Price,PlexType,Usage,Year,GarageIn,GarageOut,Units,Revenues,Style,Sqft,NbUnits,MainUnit,AdditionalCaracts,Description
0,https://www.centris.ca/fr/triplex~a-vendre~sai...,"304 - 308, Rue de Beaupré, Sainte-Anne-des-Pla...",649900,Triplex,Résidentielle seulement,1988,null,6,"1 x 3 ½, 1 x 4 ½, 1 x 5 ½",31236,Détaché,6704,3,"4 pièces, 2 chambres, 1 salle de bain",Sous-sol 6 pieds +,Description\nTriplex avec revenus de plus de 3...
1,https://www.centris.ca/fr/quadruplex~a-vendre~...,"1191 - 1191D, Rue Notre-Dame, Lavaltrie",499900,Quadruplex,Résidentielle seulement,1971,null,4,4 x 3 ½,28980,Détaché,4583,4,"3 pièces, 1 chambre, 1 salle de bain",null,Description\nQuadruplex à vendre sur la rue No...
2,https://www.centris.ca/fr/quadruplex~a-vendre~...,"825, Rue Hubert-Paré, Saint-Jean-sur-Richelieu...",649000,Quadruplex,Résidentielle seulement,1989,null,null,"2 x 3 ½, 2 x 5 ½",37920,Détaché,6000,4,"5 pièces, 3 chambres, 1 salle de bain",null,Description\nIDÉAL POUR PRORIÉTAIR OCCUPANT. B...
3,https://www.centris.ca/fr/triplex~a-vendre~sai...,"565 - 569, Rue Lapointe, Saint-Jérôme, Quartie...",625000,Triplex,Résidentielle seulement,1972,null,8,"1 x 2 ½, 1 x 3 ½, 2 x 5 ½",41520,Jumelé,6500,4,"2 pièces, 1 chambre, 1 salle de bain",Sous-sol 6 pieds +,Description\nTriplex avec revenue supplémentai...
4,https://www.centris.ca/fr/triplex~a-vendre~cha...,"201 - 203, boulevard Rousseau, Châteauguay",629000,Triplex,Résidentielle seulement,1967,2,1,"1 x 5 ½, 2 x 7 ½",30360,Détaché,5700,3,"7 pièces, 3 chambres, 1 salle de bain",Sous-sol 6 pieds +,"Description\nPrès des écoles et parcs, tout pr..."
5,https://www.centris.ca/fr/triplex~a-vendre~sai...,"663 - 667, 112e Avenue, Saint-Jérôme, Quartier...",649000,Triplex,Résidentielle seulement,1995,null,6,"2 x 4 ½, 1 x 5 ½",42000,Détaché,9057,3,"4 pièces, 2 chambres, 1 salle de bain",null,Description\nSuperbe triplex avec de bon locat...
6,https://www.centris.ca/fr/triplex~a-vendre~la-...,"520 - 524, Rue Dufort, La Prairie, Quartier Le...",649000,Triplex,Résidentielle seulement,1962,null,3,"2 x 3 ½, 1 x 5 ½",22956,Détaché,3990,3,"5 pièces, 2 chambres, 1 salle de bain",null,No Centris 24334867\nBesoin de plus de détails...
7,https://www.centris.ca/fr/triplex~a-vendre~mon...,"2775, Avenue Bilaudeau, Montréal (Mercier/Hoch...",619000,Triplex,Résidentielle seulement,1960,null,null,3 x 4 ½,32184,Jumelé,1526,3,"4 pièces, 2 chambres, 1 salle de bain",null,Description\nTriplex bien situé au coeur d'un ...
8,https://www.centris.ca/fr/triplex~a-vendre~lon...,"3600 - 3604, Grand Boulevard, Longueuil (Saint...",649000,Triplex,Résidentielle seulement,1987,null,3,3 x 4 ½,27360,Détaché,4242,3,"4 pièces, 2 chambres, 1 salle de bain","Sous-sol 6 pieds +, À proximité du REM",Description\nPremière fois sur le marché! Exce...
9,https://www.centris.ca/fr/triplex~a-vendre~lav...,"20 - 24, Rue de la Petite-Rivière, Lavaltrie",649000,Triplex,Résidentielle seulement,2009,null,6,3 x 4 ½,31800,En rangée,5212,3,"4 pièces, 2 chambres, 1 salle de bain",Sous-sol 6 pieds +,Description\nPossibilité propriétaire occupant...


In [12]:
def preprocessData(df):
    
    for index, row in df.iterrows():
        #print(row['Units'])
        sumPieces = 0
        for unit in row['Units'].split(', '):
            #print(unit)
            try:
                pieces = (re.findall(r'(?<=x )(.*)(?= ½)', unit)[0])
                if pieces.isnumeric():
                    pieces = int(pieces)
                else:
                    pieces = 0
            except:
                pieces = 0 
            nB = (re.findall(r'(?<=)(.*)(?= x)', unit)[0])
            if nB.isnumeric():
                nB = int(nB)
            else:
                nB = 0
            sumPieces += (pieces * nB)
        #print(sumPieces)
        df.at[index, 'SumUnits'] = sumPieces
        df.at[index, 'CostPerPiece'] = int(row['Price']) / sumPieces
        df.at[index, 'MRB'] = int(row['Price'])  / int(row['Revenues']) 

    return df



In [13]:
#processedDf = preprocessData(listingsInfo)
#processedDf.head()
listingsInfo.to_excel(r'C:\Users\Ricardo\Documents\Real Estate\Data\duplex_triplex_processed' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.xlsx')

In [14]:
processednewDf = preprocessData(newDF)
processednewDf.to_excel(r'C:\Users\Ricardo\Documents\Real Estate\Data\duplex_triplex_processed' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.xlsx')

C:\Users\Ricardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Ricardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi